# XE Candidate Datastore Demo

This example uses **lock** and **unlock** on both the candidate and running datastores.

In [ ]:
HOST = '127.0.0.1'
PORT_NC = 2223
USER = 'vagrant'
PASS = 'vagrant'

## Connect ncclient

In [ ]:
from ncclient import manager
from lxml import etree


def pretty_print(retval):
    print(etree.tostring(retval.data, pretty_print=True))

def my_unknown_host_cb(host, fingerprint):
    return True

m = manager.connect(host=HOST, port=PORT_NC, username=USER, password=PASS,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)

## BDI Addition Sample Edit Config

In [ ]:
config = '''<config>
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <interface>
      <BDI>
        <name>{}</name>
        <description>hello</description>
      </BDI>
    </interface>
  </native>
</config>
'''

In [ ]:
m.lock(target='running')

In [ ]:
m.lock(target='candidate')

In [ ]:
m.edit_config(config.format(666), format='xml', target='candidate', default_operation='merge')

In [ ]:
m.commit()
# m.discard_changes()

In [ ]:
m.edit_config(config.format(777), format='xml', target='candidate', default_operation='merge')

In [ ]:
m.commit()
# m.discard_changes()

**Now go check logs and output in separate windows, BEFORE any datastore is unlocked!**

### Works

In [ ]:
m.unlock(target='running')
m.unlock(target='candidate')

### Broken :(

In [ ]:
import time

m.unlock(target='candidate')
time.sleep(0.75)
m.unlock(target='running')

In [ ]:
m.close_session()

### Display An XML Diff In Text Form

In [ ]:
from difflib import context_diff

running_before_xml = etree.tostring(running_before, pretty_print=True)
running_after_xml = etree.tostring(running_after, pretty_print=True)
candidate_after_xml = etree.tostring(candidate_after, pretty_print=True)

#
# remember to skip the first few lines that have timestamps & stuff that may differ
#
print('\n'.join(context_diff(running_after_xml.decode().splitlines(),
                             candidate_after_xml.decode().splitlines())))

## Tidyup Sessions